In [1]:
import polars as pl
# %env NX_CURGAPH_AUTOCONFIG=True
import networkx as nx
import utils

DATA_PATH = "./data/"
METADATA_PATH = "./metadata/"


network = utils.SocialGraph(source_file=DATA_PATH + "social_graph.csv")
network.build(limit=1000)



dataset = utils.CandidateDataset(
    source_train=DATA_PATH + "train.csv",
    source_test=DATA_PATH + "test.csv",
    source_feature_data=METADATA_PATH + "feature_metadata_refined.json",
)


dataset.build_pipeline(limit=None, fill_missing=True, encode_categorical=True, normalize_cols=True)
dataset.build_candidate_id_set()
dataset.df_train.sort(by="ID").head().select(sorted(dataset.df_train.columns))

Successfully built & validated social graph with 1191 nodes.
Successfully instanciated Candidate Dataset and imported feature metadata.
Imported train dataset with 272819 entries (0 of which have missing features)
Imported test dataset with 48416 entries (0 of which have missing features)


ID,feature_001:1,feature_001:2,feature_001:3,feature_001:4,feature_001:5,feature_001:missing,feature_002,feature_002:missing,feature_003,feature_003:missing,feature_004,feature_004:missing,feature_005,feature_005:missing,feature_006,feature_006:missing,feature_007,feature_007:missing,feature_008,feature_008:missing,feature_009,feature_009:missing,feature_010,feature_010:missing,feature_011,feature_011:missing,feature_012:0,feature_012:1,feature_012:2,feature_012:3,feature_012:missing,feature_013,feature_013:missing,feature_014,feature_014:missing,feature_015,feature_016,feature_017,feature_017:missing,feature_018,feature_018:missing,high_conf_clean,is_cheating
u64,bool,bool,bool,bool,bool,bool,f32,bool,f32,bool,f32,bool,f32,bool,f32,bool,bool,bool,f32,bool,f32,bool,f32,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,f32,f32,f32,bool,f32,bool,bool,bool
111395511279975,false,true,false,false,false,false,-0.689304,false,-0.24793,false,0.794861,false,-1.211313,false,-1.122221,false,true,false,1.611479,false,1.935853,false,-0.001359,false,false,false,true,false,false,false,false,true,false,true,false,-0.341313,0.34603,1.369481,false,-0.122767,false,true,false
149515486601323,false,true,false,false,false,false,-0.32229,false,0.553972,false,-1.096135,false,-0.844825,false,-0.462723,false,true,false,1.361651,false,1.681633,false,-0.009491,false,false,false,true,false,false,false,false,false,false,true,false,-0.344136,-0.432933,-1.716514,true,1.101163,false,true,false
160906985161287,false,false,false,false,true,false,1.145764,false,0.954923,false,-1.096135,false,1.354103,false,0.856274,false,true,false,1.361651,false,1.681633,false,-0.000379,false,false,false,true,false,false,false,false,true,false,true,false,-0.329906,-0.121348,-0.920808,false,-0.454953,false,false,false
372923524293330,false,false,false,false,false,true,-1.056318,true,-1.851734,true,-0.285708,false,0.621127,false,0.856274,false,true,false,1.611479,false,1.681633,false,0.000249,false,false,false,true,false,false,false,false,true,false,true,false,-0.344038,-0.27714,-1.545096,false,0.485668,false,true,false
396236678209396,false,false,false,true,false,false,0.411737,false,0.954923,false,0.794861,false,0.254639,false,0.196776,false,true,false,1.361651,false,-0.606347,false,-0.007269,false,false,false,true,false,false,false,false,true,false,true,false,-0.331746,-0.121348,0.413475,false,0.847724,false,true,false


In [2]:
dataset.df_train.sort(by="ID")

ID,feature_002,feature_003,feature_004,feature_005,feature_006,feature_007,feature_008,feature_009,feature_010,feature_011,feature_013,feature_014,feature_015,feature_016,feature_017,feature_018,high_conf_clean,is_cheating,feature_007:missing,feature_011:missing,feature_013:missing,feature_014:missing,is_cheating:missing,feature_002:missing,feature_003:missing,feature_004:missing,feature_005:missing,feature_006:missing,feature_008:missing,feature_009:missing,feature_010:missing,feature_017:missing,feature_018:missing,feature_001:1,feature_001:2,feature_001:3,feature_001:4,feature_001:5,feature_001:missing,feature_012:0,feature_012:1,feature_012:2,feature_012:3,feature_012:missing
u64,f32,f32,f32,f32,f32,bool,f32,f32,f32,bool,bool,bool,f32,f32,f32,f32,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
111395511279975,-0.689304,-0.24793,0.794861,-1.211313,-1.122221,true,1.611479,1.935853,-0.001359,false,true,true,-0.341313,0.34603,1.369481,-0.122767,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,true,false,false,false,false
149515486601323,-0.32229,0.553972,-1.096135,-0.844825,-0.462723,true,1.361651,1.681633,-0.009491,false,false,true,-0.344136,-0.432933,-1.716514,1.101163,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,true,false,false,false,false,true,false,false,false,false
160906985161287,1.145764,0.954923,-1.096135,1.354103,0.856274,true,1.361651,1.681633,-0.000379,false,true,true,-0.329906,-0.121348,-0.920808,-0.454953,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,true,false,false,false,false
372923524293330,-1.056318,-1.851734,-0.285708,0.621127,0.856274,true,1.611479,1.681633,0.000249,false,true,true,-0.344038,-0.27714,-1.545096,0.485668,true,false,false,false,false,false,false,true,true,false,false,false,false,false,false,false,false,false,false,false,false,false,true,true,false,false,false,false
396236678209396,0.411737,0.954923,0.794861,0.254639,0.196776,true,1.361651,-0.606347,-0.007269,false,true,true,-0.331746,-0.121348,0.413475,0.847724,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,true,false,false,false,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
18446539283039586544,1.512778,0.553972,0.524719,-0.844825,0.856274,true,1.611479,1.681633,-0.001608,false,true,true,1.414438,0.034445,-1.285127,0.586319,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,true,false,false,false,false
18446564111536718883,1.512778,-0.648881,1.065004,0.987615,0.856274,true,-0.886794,-0.606347,-0.010341,false,true,true,-0.344186,-0.27714,-0.062558,-1.071172,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,true,false
18446710016529126028,1.512778,-1.049832,1.065004,0.621127,-1.45197,true,-0.636967,-0.606347,-0.008471,false,false,false,-0.329667,-0.27714,-1.401475,-0.137792,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,true,false,false


In [2]:
df_train, df_test = dataset.split()

print(type(df_train))
df_train.head()


Split dataframe into 218255 training and 54564 validation entries.
<class 'polars.dataframe.frame.DataFrame'>


ID,feature_002,feature_003,feature_004,feature_005,feature_006,feature_007,feature_008,feature_009,feature_010,feature_011,feature_013,feature_014,feature_015,feature_016,feature_017,feature_018,high_conf_clean,is_cheating,feature_001:1,feature_001:2,feature_001:3,feature_001:4,feature_001:5,feature_001:missing,feature_012:0,feature_012:1,feature_012:2,feature_012:3,feature_012:missing
u64,i16,i16,i16,i16,i16,bool,i16,i16,f32,bool,bool,bool,f32,f32,f32,f32,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
9925137249666252685,7,1,9,6,0,false,0,0,0.0,false,false,false,0.023368,0.0,0.0,0.145,false,true,false,false,false,false,true,false,false,false,false,true,false
17510260092577491870,7,4,10,2,7,true,1,0,0.0,false,true,true,0.022894,1.0,15.95,0.33,false,false,false,false,false,true,false,false,true,false,false,false,false
5956214523393149507,2,4,8,2,6,false,0,0,0.0,false,false,true,0.02588,1.0,15.35,0.3,true,false,false,true,false,false,false,false,true,false,false,false,false
1148513036740754477,7,6,1,9,7,true,0,0,0.0,false,true,true,0.035694,3.0,11.83354,0.3,false,false,false,false,false,true,false,false,false,true,false,false,false
13647347889513101289,2,8,1,9,6,false,0,0,0.0,false,true,true,0.059769,1.0,8.6,0.72,true,false,true,false,false,false,false,false,true,false,false,false,false


In [3]:
params = {
    "objective": "binary:logistic",
    "tree_method": "hist",
    "booster": "gbtree",
    "eval_metric": "auc",
    "n_estimators": 20,
    "max_depth": 8,
    "subsample": 0.5,
    "colsample_bytree": 1.0,
    "colsample_bylevel": 0.6,
    "colsample_bynode": 1.0,
    "verbosity": 2,
    "device": "cpu",
    "enable_categorical": True,
    "n_jobs": 4,
}

xgb = dataset.fit_xgboost_model(df_train)

dataset.evaluate_xgboost_model(xgb, df_test)

XGBoost model trained successfully!
Evaluation metrics -> LogLoss: 0.24553, Accuracy: 0.8969


/home/matthias/miniforge3/envs/playground/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:49:42] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "n_estimators" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


{'logloss': 0.24553314688474925,
 'accuracy': 0.8969283776849205,
 'raw_eval': '[0]\teval-logloss:0.24553314688474925'}

In [3]:
dataset.compute_isolate_candidates_from_network(network)

There are 87878 elements in the dataset which are not present in the network.


In [5]:
nb_connected_components = nx.number_connected_components(network.graph)  #, backend="cugraph"
nb_connected_components

17624

In [ ]:
from scipy.sparse import csgraph
A = nx.to_scipy_sparse_array(
            network.graph, nodelist=network.graph.nodes, dtype=float
        ).tocsr()
L = csgraph.laplacian(A, normed=True)

print(f"Normalized Laplacian matrix L has shape {L.shape} with {L.nnz} non-zero entries.")

Normalized Laplacian matrix L has shape (1727366, 1727366) and 5146954 non-zero entries.


In [30]:
import numpy as np
D = L - L.T
D.nnz, np.allclose(D.data, 0)

(16252, True)

In [ ]:
from scipy.sparse.linalg import eigsh

k = 2  # number of eigenvectors
vals, vecs = eigsh(L, k=k, which='LM', tol=1e-6, maxiter=10)

In [39]:
import cupyx.scipy.sparse as cusp
from cupyx.scipy.sparse.linalg import eigsh

L_gpu = cusp.csr_matrix(L)

vals, vecs = eigsh(L_gpu, k=2, which='LM', tol=1e-6, maxiter=1000)
vecs

array([[-7.51713730e-03, -1.62070089e-03],
       [ 7.17042885e-04,  1.35433152e-04],
       [ 5.76652349e-04,  4.13867316e-04],
       ...,
       [ 1.39767702e-05, -6.34658955e-04],
       [-1.24909278e-05, -5.22619029e-05],
       [ 1.37553979e-04, -2.73080792e-04]], shape=(1727366, 2))

Perfect. You already have the sparse adjacency `A`. Here's a concrete, practical path.

---

### 1. Construct the Laplacian

For an unnormalized Laplacian:

```python
from scipy.sparse import csgraph

L = csgraph.laplacian(A, normed=False)  # shape (2_000_000, 2_000_000)
```

For a normalized Laplacian:

```python
L_norm = csgraph.laplacian(A, normed=True)
```

`normed=True` is often better for embeddings.

---

### 2. Solve for the first k eigenpairs

#### CPU + SciPy (sparse, symmetric)

```python
from scipy.sparse.linalg import eigsh

k = 20  # number of eigenvectors
# 'SM' = smallest magnitude eigenvalues
vals, vecs = eigsh(L_norm, k=k, which='SM')
```

**Notes**:

* `eigsh` uses ARPACK internally.
* For large graphs, `k` should be small (<=100).
* `L_norm` must be symmetric (it is).
* Runtime ~ O(iterations * edges).

Optional speedup: precondition with LOBPCG.

```python
from scipy.sparse.linalg import lobpcg
import numpy as np

X = np.random.rand(L.shape[0], k)  # initial guess
vals, vecs = lobpcg(L_norm, X)
```

With `pyamg` you can add a multigrid preconditioner:

```python
import pyamg
ml = pyamg.ruge_stuben_solver(L_norm)
M = ml.aspreconditioner()
vals, vecs = lobpcg(L_norm, X, M=M)
```

---

### 3. GPU options

If you want CUDA:

* **cuSolver/cuSparse** via `cupy` or `cugraph`.
* Example with CuPy:

```python
import cupy as cp
import cupyx.scipy.sparse as cusparse
from cupyx.scipy.sparse.linalg import eigsh

A_gpu = cusparse.csr_matrix(A)
L_gpu = cusparse.csgraph.laplacian(A_gpu, normed=True)

vals, vecs = eigsh(L_gpu, k=20, which='SM')
```

**Notes**:

* GPU memory must hold the sparse Laplacian + Krylov basis.
* Works well for k up to a few dozen on typical GPUs.

---

### Recommendation

* **Exact CPU, moderate k**: `eigsh` or `lobpcg + pyamg`.
* **GPU speedup**: `cupyx.scipy.sparse.linalg.eigsh`.
* **Approximate, huge graphs**: Nyström or randomized methods (scikit-learn `TruncatedSVD` on normalized adjacency).

---

If you want, I can write a **ready-to-run pipeline for 2M nodes** that chooses CPU or GPU depending on memory and computes the first k eigenvectors efficiently. It will include preconditioning for LOBPCG. Do you want me to do that?


In [ ]:
result = nx.betweenness_centrality(network.graph, k=10)